In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.misc
import matplotlib.pyplot as plt
from random import randint
from scipy import misc

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Haohan's preprocessing functions

def threshold_(img, thres=250):
    img[img > thres] = 255
    img[img <= thres] = 0
    return img


def update_bound(i, j, bound):
    if i < bound[0]:
        bound[0] = i
    if i > bound[1]:
        bound[1] = i
    if j < bound[2]:
        bound[2] = j
    if j > bound[3]:
        bound[3] = j
    return bound


def find(x, i, j, record, bound=None):
    # s_h, e_h, s_w, e_w = 0, 0, 0, 0
    if i < 0 or i > 63 or j < 0 or j > 63:
        return bound

    if record[i][j] > 0.5 or x[i][j] < 240:
        return bound

    record[i][j] = 1

    if bound is None:
        bound = [0, 0, 0, 0]

    bound = update_bound(i, j, bound)

    step = [[1, 0], [0, -1], [-1, 0], [0, 1]]
    for step_ in step:
        find(x, i + step_[0], j + step_[1], record, bound)

    return bound


def detect(x):
    record = np.zeros(x.shape, dtype=np.int32)
    max_area = 0
    max_bound = None
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if record[i][j] > 0.5 or x[i][j] < 240:
                continue
            bound = [i, i, j, j]
            bound = find(x, i, j, record, bound)
            if bound is not None:
                # cur_area = (bound[1] - bound[0]) * (bound[3] - bound[2])
                cur_area = max(bound[1] - bound[0], bound[3] - bound[2])
                if cur_area > max_area:
                    max_area = cur_area
                    max_bound = bound

    # print bound
    padding = 2

    # padding the bound
    max_bound[0] = max_bound[0] - padding if max_bound[0] - padding > 0 else 0
    max_bound[1] = max_bound[1] + padding if max_bound[1] + padding < 64 else 64
    max_bound[2] = max_bound[2] - padding if max_bound[2] - padding > 0 else 0
    max_bound[3] = max_bound[3] + padding if max_bound[3] + padding < 64 else 64

    return x[max_bound[0]: max_bound[1], max_bound[2]: max_bound[3]]


def padding(image):
    img_size = (32, 32)
    h, w = image.shape
    pad_h = int((max(h, w) - h) / 2)
    pad_w = int((max(h, w) - w) / 2)
    new_image = np.zeros([h + 2 * pad_h, w + 2 * pad_w], dtype=np.float32)
    new_image[pad_h: pad_h + h, pad_w: pad_w + w] = image
    new_image = misc.imresize(new_image, img_size)
    return new_image

def preprocess(image): 
    return threshold_(padding(detect(image)))

In [ ]:
# Haohan's model

# utilities 

def softmax(output_array):
    logits_exp = np.exp(output_array)
    return logits_exp / np.sum(logits_exp, axis=1, keepdims=True)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    # derivative of the sigmoid function
    return -1 * np.exp(-x) / ((1+np.exp(-x)) ** 2)

def categorical_crossentropy(output, target):
    assert output.shape[1] == target.shape[1], "model output" + str(output.shape) + \
                                               " should have same shape with target" + str(target.shape)
    # output /= np.sum(output, axis=len(output.shape) - 1, keepdims=True)
    output = softmax(output)
    # manual computation of crossentropy
    epsilon = np.finfo(float).eps
    output = np.clip(output, epsilon, 1. - epsilon)
    return np.mean(- np.sum(target * np.log(output), axis=len(output.shape) - 1)), target - output


# layer classes

class Layer:
    def __init__(self):
        pass
    def forward(self, b_input):
        pass
    def backward(self, loss, lr):
        pass

class Sigmoid(Layer):
    def __init__(self):
        Layer.__init__(self)
        
    def forward(self, b_input):
        return sigmoid(b_input)
    
    def backward(self, loss, lr):
        return loss * sigmoid_derivative(loss)

class FC(Layer):
    def __init__(self, units):
        Layer.__init__(self)
        self.w = None
        self.b = np.random.randn(1, units)
        self.output = 0
        self.b_input = None
        self.units = units

    def forward(self, b_input):
        self.b_input = b_input
        if self.w is None:
            self.w = np.random.randn(self.b_input.shape[1], self.units)
        z1 = self.b_input.dot(self.w) + self.b
        # self.output = sigmoid(z1)
        return z1

    def backward(self, loss, lr=0.001, reg_lambda=0.0001):
        dw = self.b_input.T.dot(loss)
        dw += reg_lambda * self.w  # add penalty

        db = np.sum(loss, axis=0, keepdims=True)

        top_loss = loss.dot(self.w.T)

        self.w += lr * dw
        self.b += lr * db

        return top_loss
    
    
# main model class

class Fann_Model:
    def __init__(self, input_shape):
        self.layers = []
        self.outputs = []
        self.gradient = []
        self.lr = 0.001
        self.input = None

    def add(self, layer):
        self.layers.append(layer)
        self.outputs.append([])
        self.gradient.append([])

    def forward_all(self):
        for i, layer in enumerate(self.layers):
            if i == 0:
                self.outputs[i] = layer.forward(self.input)
                continue
            self.outputs[i] = layer.forward(self.outputs[i - 1])

    def back_propagation(self):
        for i, layer in reversed(list(enumerate(self.layers))):
            self.gradient[i] = layer.backward(self.gradient[i + 1], self.lr)

    def get_accuracy(self, output, target):
        # print output
        # print target
        _ = np.array([i.argmax() for i in output]).T
        y = np.array([i.argmax() for i in target]).T

        total = len(output)
        right = (_ == y).sum()

        return float(right) / float(total)

    def fit(self, x, y, max_epoch=5, lr=0.0001):
        self.input = x
        self.lr = lr
        if len(self.gradient) == len(self.outputs):
            self.gradient.append([])
            #print(len(self.outputs), len(self.gradient))

        assert len(x.shape) == 2, "X should be in 2 dimensions like (40, 2)"
        assert len(y.shape) == 2, "y should be in 2 dimensions like (40, 1)"

        losses = []
        for epoch in range(max_epoch):
            # forward
            self.forward_all()
            loss, gradient = categorical_crossentropy(self.outputs[-1], y)

            accuracy = self.get_accuracy(self.outputs[-1], y)
            #print("Current epoch:", epoch, "Loss:", loss, "accuracy:", accuracy)
            losses.append(loss)

            self.gradient[-1] = gradient

            # backward
            self.back_propagation()

        return losses

    def predict(self, x):
        assert len(x.shape) == 2, "X should be in 2 dimensions like (40, 2)"
        self.input = x
        self.forward_all()
        return self.outputs[-1]

    def evaluate(self, x, y):
        pred = self.predict(x)
        accuracy = self.get_accuracy(pred, y)
        return accuracy

In [ ]:
# make training and validation sets
# k is the number of sets

def crossValidation(X, y, k=10): 
    X_train = []
    X_valid = []
    y_train = []
    y_valid = []

    fold_length = int (len(X) / k)
    for i in range(k):
        fold_start = i*fold_length
        fold_end = fold_start + fold_length
        X_valid.append(X[fold_start:fold_end])
        X_train.append(np.concatenate((X[:fold_start], X[fold_end:])))
        y_valid.append(y[fold_start:fold_end])
        y_train.append(np.concatenate((y[:fold_start], y[fold_end:])))
        
    return X_train, X_valid, y_train, y_valid

In [ ]:
# create the model and build the layers
# the hyperparameters are: num_folds, num_layers, num_neurons, num_epochs, learning_rate

def runModel(X, y, num_folds, num_layers=2, num_neurons=10, num_epochs=100, learning_rate=0.001): 
    
    X_train, X_valid, y_train, y_valid = crossValidation(X, y, k=num_folds)
    
    model = Fann_Model(X.shape)
    for i in range(num_layers - 1): 
        model.add(FC(num_neurons))
        model.add(Sigmoid())
    # the last layer has as many units as there are classes
    model.add(FC(classes))
    model.add(Sigmoid())
    
    # train the model
    validation_scores = []
    for i in range(num_folds): 
        #print("\nTraining with fold", i)
        model.fit(X_train[i], y_train[i], max_epoch=num_epochs, lr=learning_rate)
        score = model.evaluate(X_valid[i], y_valid[i])
        #print("Validation score for fold ", i, ":", score)
        validation_scores.append(score)

    mean_score = np.mean(validation_scores)
    #print("\nAverage validation score:", mean_score)
    return mean_score
    

In [ ]:
# input data

X_total = np.loadtxt("../input/train_x.csv", delimiter=",") # load from text 
y_total = np.loadtxt("../input/train_y.csv", delimiter=",") 
X_total = X_total.reshape(-1, 64, 64) # reshape 
y_total = y_total.reshape(-1, 1) 

In [ ]:
# preparation

# here I get smaller subsets for testing
X = X_total[:300]
y = y_total[:300]

# 1-of-10 categorization for the classes
from keras.utils import to_categorical
classes = 10
y = to_categorical(y, num_classes=classes)

# image preprocessing 
X = np.array([preprocess(x) for x in X])

# reshaping the input array in 2D
#print(X.shape)
X = X.reshape(X.shape[0], -1)
#print(X.shape)



In [ ]:
# Running the model

# hyperparameters
folds = 10
layers = 2  #number of layers in the network
neurons = 10 #number of units per layer (except the last layer)
epochs = 10  #number of epochs for training
lr = 0.001 #learning rate

#validation_result = runModel(X, y, num_folds=folds, num_layers=layers, num_neurons=neurons, num_epochs=epochs, learning_rate=lr)
#print("\nAverage validation score:", validation_result)

# Loop with several values of hyperparameters to find the best combination.
# WARNING: do not put ranges that are too large, otherwise this will take forever. 
# It's better to fix most hyperparameters and vary only one of them. 
# An asterisk in the output indicates the best validation score so far in the current run.
best_result = 0
for layers in range(2, 6):
    for neurons in range(2, 3): 
        for epochs in range(50, 60, 10):
            for lr in np.logspace(-5, -2, num=4): 
                validation_result = runModel(X, y, num_folds=folds, num_layers=layers, num_neurons=neurons, num_epochs=epochs, learning_rate=lr)
                asterisk = ""
                if validation_result > best_result: 
                    best_result = validation_result
                    asterisk = "*"
                print("%sAverage validation score with folds=%s, num_layers=%s, num_neurons=%s, num_epochs=%s, learning_rate=%s: %s" % (asterisk, folds, layers, neurons, epochs, lr, validation_result))

